## Classificação da Intenção

Para classificar a intenção decidimos utilizar a técnica de bagging.

In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

train = pd.read_csv('/home/jupyter/hackathon_petrobras/mensagem_entidade.txt', sep=';')

In [4]:
train.head(4) #primeiros 4 registros

,mensagem,intencao
0,quantos Marcos críticos Estão atrasados no ped...,marcos_criticos
1,para novembro quantos são os Marcos críticos p...,marcos_criticos
2,Quais são os Marcos críticos previstos para o ...,marcos_criticos
3,quantos Marcos críticos estão previstos para s...,marcos_criticos


In [5]:
train.shape #numero de registros do dataset

(1253, 2)

In [9]:
train['intencao'].unique()#tipos diferentes de labels

array(['marcos_criticos', 'gog', 'obrigacao', 'capex'], dtype=object)

Precisamos normalizar o texto. Remover ascentos e deixar todas a palavras em minúsculo.

In [7]:
from unicodedata import normalize

lower_case = normalize('NFKD', train["mensagem"][0]).encode('ASCII', 'ignore').decode('ASCII').lower() #letras minúsculas
words = lower_case.split()
print(train["mensagem"][0])
print(lower_case)

NameError: name 'train' is not defined

Temos palavras que são muito comuns em textos, mas não represetam um grandes significados na semântica, que geralmente são artigos, pronomes e preposições, então chamamos essas palavras de "stop words", que no inglês temos palavras de exemplo: "a", "and", "is", e "the". Por coviniência temos pacotes no Python que possuem uma lista de palavras consideradas como "stop words" e também realizam esse filtro, para realizar esse processamento. temos o pacote Python Natural Language Toolkit (NLTK) que também possue muito outras ferramentas para lidar com problemas de processamento de linguagem natual.

In [26]:
#import nltk
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from nltk.corpus import stopwords #lista de stop words da lingua portuguesa
print (stopwords.words("portuguese"))

['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estivermos

In [5]:
stop = stopwords.words("portuguese") #criar um array com as stop words do português

In [9]:
# não é possivel remover os assuntos em uma lista, então vamos percorrer o array aplciando emc ada registro
lista_stop = [normalize('NFKD', x).encode('ASCII', 'ignore').decode('ASCII') for x in stop]  

In [10]:
lista_stop

['de',
 'a',
 'o',
 'que',
 'e',
 'do',
 'da',
 'em',
 'um',
 'para',
 'com',
 'nao',
 'uma',
 'os',
 'no',
 'se',
 'na',
 'por',
 'mais',
 'as',
 'dos',
 'como',
 'mas',
 'ao',
 'ele',
 'das',
 'a',
 'seu',
 'sua',
 'ou',
 'quando',
 'muito',
 'nos',
 'ja',
 'eu',
 'tambem',
 'so',
 'pelo',
 'pela',
 'ate',
 'isso',
 'ela',
 'entre',
 'depois',
 'sem',
 'mesmo',
 'aos',
 'seus',
 'quem',
 'nas',
 'me',
 'esse',
 'eles',
 'voce',
 'essa',
 'num',
 'nem',
 'suas',
 'meu',
 'as',
 'minha',
 'numa',
 'pelos',
 'elas',
 'qual',
 'nos',
 'lhe',
 'deles',
 'essas',
 'esses',
 'pelas',
 'este',
 'dele',
 'tu',
 'te',
 'voces',
 'vos',
 'lhes',
 'meus',
 'minhas',
 'teu',
 'tua',
 'teus',
 'tuas',
 'nosso',
 'nossa',
 'nossos',
 'nossas',
 'dela',
 'delas',
 'esta',
 'estes',
 'estas',
 'aquele',
 'aquela',
 'aqueles',
 'aquelas',
 'isto',
 'aquilo',
 'estou',
 'esta',
 'estamos',
 'estao',
 'estive',
 'esteve',
 'estivemos',
 'estiveram',
 'estava',
 'estavamos',
 'estavam',
 'estivera',
 'es

In [18]:
words=[w for w in words if not w in lista_stop] 
print(words)

['quantos', 'marcos', 'criticos', 'atrasados', 'pedido', 'ano', 'corrente']


In [11]:
def audio_to_words(raw_review):
    letters_numbers = normalize('NFKD', raw_review).encode('ASCII', 'ignore').decode('ASCII')
    words = letters_numbers.lower().split()
    meaningful_words=[w for w in words if not w in lista_stop]
    return(' '.join(meaningful_words))

In [12]:
#testando a função
clean_audio = audio_to_words(train['mensagem'][0])
print(clean_audio)

quantos marcos criticos atrasados pedido ano corrente


In [13]:
num_audios = train['mensagem'].size

In [14]:
clean_train_audio=[]
for i in range(0, num_audios):
    clean_train_audio.append(audio_to_words(train['mensagem'][i]))

In [15]:
clean_train_audio

['quantos marcos criticos atrasados pedido ano corrente',
 'novembro quantos marcos criticos previstos',
 'quais marcos criticos previstos proximo mes',
 'quantos marcos criticos previstos setembro 2019 pr',
 'quantos marcos criticos realizados pdf mes abril',
 'quantos marcos criticos realizados pdf 2018',
 'quantas marcos fisicos realizados marco 2019 pdf',
 'quantos marcos criticos atrasados 2019',
 'quantas marcos fisicos docentes realizados',
 'quantos marcos criticos previstos ano 2019 pdf',
 'realizacao indicador marcos critica',
 'quantos marcos criticos gerente geral sonhos possuem',
 'gerencia pdf quantos marcos criticos realizados massa 2019',
 'quantos marcos criticos pedestre atraso dois mil dezenove',
 'quais marcos criticos pdrn realizados neste mes',
 'previsao realizacao marcos criticos rn mes junho',
 'quantos marcos criticos gerencia pedido possuem 2019',
 'quantos marcos criticos concluidos pdp',
 'total inscritos planejados 2019 cpf',
 'quantos marcos criticos prev

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
# configurar os parametros do WordtoVec/Tokeninzação e criar o objeto
vectorizer = CountVectorizer(analyzer='word',tokenizer=None,preprocessor = None, stop_words = None,max_features = 7000)
# aplicar WordtoVec/Tokeninzação
train_data_features=vectorizer.fit_transform(clean_train_audio)
# aplicar a estrutura de dados numpy array
train_data_features=train_data_features.toarray()

In [27]:
# vocabulario de todas das palavaras mais importantes de todos os audios
vcab=vectorizer.get_feature_names()
print(vcab)

['000', '10', '100', '1018', '1019', '17', '18', '19', '200', '2000', '2016', '2017', '2018', '2019', '21', 'ab', 'abril', 'acumulada', 'acumulado', 'acumular', 'adsense', 'agencia', 'agora', 'agosto', 'agua', 'ai', 'alcatex', 'alex', 'anel', 'ano', 'antt', 'anual', 'aplicacao', 'apps', 'aqui', 'area', 'ariel', 'assistente', 'atendimento', 'atingimento', 'atos', 'atrasada', 'atrasado', 'atrasados', 'atraso', 'atual', 'atualizada', 'bar', 'barcos', 'bebe', 'blog', 'blogue', 'bode', 'brincar', 'bug', 'cabo', 'caceres', 'cafe', 'cap', 'capa', 'capacete', 'capacetes', 'capas', 'capes', 'capeta', 'capex', 'capim', 'capinhas', 'capistrano', 'capital', 'capitao', 'capitulo', 'capitulos', 'capo', 'caprichando', 'caps', 'caput', 'carpete', 'carpetes', 'carro', 'catequizando', 'catete', 'catetos', 'catex', 'cdp', 'cemps', 'cenario', 'center', 'centers', 'centro', 'centros', 'cep', 'clima', 'clinicas', 'cog', 'comemorado', 'concluidas', 'concluidos', 'coobrigacao', 'correalizacao', 'corrente', 'c

### Random Forest

Como de prática na técnica de bagging, utilizamos o algoritimo de Random Forest.

In [18]:
from sklearn.model_selection import train_test_split
# Split dos dados em treino e validação
X_train, X_test, y_train, y_test = train_test_split(train_data_features, train["intencao"], test_size=0.2, random_state=42)

In [29]:
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(n_estimators=100, random_state=42)
%time forest = forest.fit( X_train, y_train )

CPU times: user 164 ms, sys: 4 ms, total: 168 ms
Wall time: 164 ms


In [30]:
result = forest.predict(X_test)

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, result)

0.9760956175298805

### Regressão logarítmica

In [33]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=42, solver='lbfgs')

In [34]:
%time clf = clf.fit( X_train, y_train )
result3 = clf.predict(X_test)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


CPU times: user 312 ms, sys: 0 ns, total: 312 ms
Wall time: 270 ms


In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, result3)

0.9760956175298805

### Testando o modelo

In [36]:
word = "como que a capex decorreu no ano de 2019"

In [46]:
# o modelo só aceita entrada array ou dataframe, portanto vamos criar uma também para o teste
entrada=[]
entrada.append(audio_to_words(word))
entrada.append(word)

In [41]:
print(entrada)

['como que a capex decorreu no ano de 2019']


In [43]:
# aplicamos a tokenização do texto teste
sample_final=vectorizer.transform(entrada)
# aplicamos array numpy
sample_final=sample_final.toarray()

In [36]:
def predict_coment(word):
    to_array=[]
    to_array.append(audio_to_words(word))
    sample_final=vectorizer.transform(to_array)
    sample_final=sample_final.toarray()
    result = forest.predict(sample_final)    
    return result#, word

In [38]:
predict_coment('como que a capex decorreu no ano de 2019')[0]

'capex'

In [35]:
reultado= predict_coment('como que a capex decorreu no ano de 2019')[0]


In [52]:
predict_coment('os marcos antes de fevereiro inviabilizaram o processo')

(array(['marcos_criticos'], dtype=object),
 'os marcos antes de fevereiro inviabilizaram o processo')

In [51]:
predict_coment('Como vamos prever o gog do ano seguinte')

(array(['gog'], dtype=object), 'Como vamos prever o gog do ano seguinte')

In [53]:
predict_coment('qual a realização da anel no mês que vem')

(array(['obrigacao'], dtype=object),
 'qual a realização da anel no mês que vem')